N = 4 <br>
prototype

<h4>Item <i>"a)"</i> Onde f = 0 e u(x,y)= alpha * np.exp(x) * np.sin(y) na fronteira</h4>

In [10]:
import numpy as np

In [27]:
N=2**3
MAXITER=10**3

alpha = 3.0
h=1./N

TOL = 10**-5 * h

<h3>Funções Auxiliares</h3>

<h4>Jacobi</h4>

In [28]:
def jacobi_method(U_old, U_new, h, F):
    for i in range(1,N):
        for j in range(1,N):
            U_new[i][j] = 0.25*(U_old[i-1][j] + U_old[i+1][j] + U_old[i][j-1] + U_old[i][j+1] + h**2 * F[i][j])

<h4>PASSO 1 e 2</h4>

In [29]:
U_old = np.zeros((N+1,N+1))
U_new = np.zeros((N+1,N+1))
F = np.zeros((N+1,N+1)) 

<h4>PASSO 3</h4>

In [30]:
F = np.zeros((N+1,N+1)) 

<h4>PASSO 4</h4>

In [31]:
# Atualiza as bordas


# Parte inferior e superior da Malha
for i in range(N+1):
    U_old[i, 0] = alpha * np.exp(h*i) * np.sin(h*0.)
    U_old[i, N] = alpha * np.exp(h*i) * np.sin(h*N)

# Parte esquerda e direita da Malha
for j in range(N+1):
    U_old[0, j] = alpha * np.exp(h*0.) * np.sin(h*j)
    U_old[N, j] = alpha * np.exp(h*N) * np.sin(h*j)    


In [32]:
U_old

array([[0.        , 0.3740242 , 0.74221188, 1.09881759, 1.43827662,
        1.75529182, 2.04491628, 2.30263051, 2.52441295],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 2.86053463],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 3.2414104 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 3.67299918],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 4.16205333],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 4.7162243 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 5.34418227],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 6.05575187],


<h4>PASSO 5-7</h4>

In [33]:
for it in range(MAXITER):
    jacobi_method(U_old, U_new, h, F)
    
#     if calculate_diff(U_new, U_old) <= TOL:
#         print('Algoritmo convergiu com %d iterações' %(it))
#         break
    
    U_old = U_new.copy()

In [34]:
U_new

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 1.79103248e-36, 3.57911721e-36, 4.32393491e-36,
        5.06163610e-36, 4.32393491e-36, 3.57911721e-36, 1.79103248e-36,
        0.00000000e+00],
       [0.00000000e+00, 3.57911721e-36, 6.11496740e-36, 8.64075330e-36,
        8.64786983e-36, 8.64075330e-36, 6.11496740e-36, 3.57911721e-36,
        0.00000000e+00],
       [0.00000000e+00, 4.32393491e-36, 8.64075330e-36, 1.04389023e-35,
        1.22198705e-35, 1.04389023e-35, 8.64075330e-36, 4.32393491e-36,
        0.00000000e+00],
       [0.00000000e+00, 5.06163610e-36, 8.64786983e-36, 1.22198705e-35,
        1.22299348e-35, 1.22198705e-35, 8.64786983e-36, 5.06163610e-36,
        0.00000000e+00],
       [0.00000000e+00, 4.32393491e-36, 8.64075330e-36, 1.04389023e-35,
        1.22198705e-35, 1.04389023e-35, 8.64075330e-36, 4.32393491e-36,
        0.0